In [6]:
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io

from bs4 import BeautifulSoup
domain = "nba.com"
wordsearch = ["celtics","nets","knicks","raptors","76ers"]
index_list = ["2019-04","2019-09","2019-13"]
def search_domain(domain):

    record_list = []
    
    print("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added %d results." % len(records))
            
    
    print("[*] Found a total of %d hits." % len(record_list))
    
    return record_list  

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""
    htmlData = []
    #print(len(data))
    if len(data):
            htmlData = data.decode('UTF-8').strip().split("\r\n\r\n")
            if(len(htmlData)>2):
                response = htmlData[2]
    return response
count=0
weburlTest=[]
mainTopic = "news"
searchMain = "/"+mainTopic+"/"
record_list = search_domain(domain)
index = open('atlantic_crawlData.txt', 'w')
for record in record_list:
    for idx in range(0,len(wordsearch)):
            wordsearcht = wordsearch[idx]
            keyword = "/"+wordsearcht+"/"
            if(searchMain.lower() in record['url']):
                if(keyword.lower() in record['url']):
                    print(record['url'])
                    count+=1
                    html_content = download_page(record)
                    soup = BeautifulSoup(html_content)
                    for p in soup.find_all("p", text=True):
                        rete = p.get_text()
                        rete = rete.replace('\n', '')
                        rete = rete.replace('\r\n', '')
                        rete = rete.replace('\\n','')
                        rete = rete.replace('\t','')
                        #print(rete)
                        index.write(rete)
                    index.write("\n") 
print(count)

[*] Trying target domain: nba.com
[*] Trying index 2019-04
[*] Added 13445 results.
[*] Trying index 2019-09
[*] Added 14007 results.
[*] Trying index 2019-13
[*] Added 14261 results.
[*] Found a total of 41713 hits.
https://www.nba.com/celtics/news/post-ups
https://www.nba.com/celtics/news/post-ups-20181105
https://www.nba.com/celtics/news/post-ups-20181108
https://www.nba.com/celtics/news/post-ups-20181109
https://www.nba.com/celtics/news/post-ups-20190109
https://www.nba.com/celtics/news/post-ups-20190110
https://www.nba.com/celtics/news/press_release/celtics-complete-trade-brooklyn-nets
https://www.nba.com/celtics/news/press_release/celtics-waive-milicic
http://www.nba.com/celtics/news/press_release/olynyk-chosen-all-rookie-second-team
https://www.nba.com/celtics/news/press_release/olynyk-chosen-all-rookie-second-team
https://www.nba.com/celtics/news/pressrelease/celtics-complete-trade-oklahoma-city
https://www.nba.com/celtics/news/pressrelease/celtics-re-sign-aron-baynes
https://w

http://www.nba.com/celtics/news/sidebar/top-prospects-eye-boston-landing-spot
https://www.nba.com/celtics/news/sidebar/top-prospects-eye-boston-landing-spot
https://www.nba.com/celtics/news/twhs-20180621-draft
https://www.nba.com/amp/nets/news/2018/09/18/rodions-kurucs-built-his-road-to-brooklyn-from-a-family-tree
http://www.nba.com/celtics/news/notes/notes110207-pierce-emotional.html?rss=true
https://www.nba.com/celtics/news/notes/notes110207-pierce-emotional.html?rss=true
http://www.nba.com/celtics/news/post-ups
https://www.nba.com/celtics/news/post-ups
https://www.nba.com/celtics/news/post-ups-20171127
https://www.nba.com/celtics/news/post-ups-20180328
https://www.nba.com/celtics/news/post-ups-20180331
https://www.nba.com/celtics/news/post-ups-20180930
https://www.nba.com/celtics/news/post-ups-20181002
https://www.nba.com/celtics/news/post-ups-20181006
https://www.nba.com/celtics/news/post-ups-20181130
https://www.nba.com/celtics/news/post-ups-20181201
https://www.nba.com/celtics/ne

https://www.nba.com/celtics/news/sidebar/shoot-visitor-020819-rondo-admits-its-little-strange-returning-boston-laker
https://www.nba.com/celtics/news/sidebar/shoot-visitor-021418-reunited-ex-celtics-bradley-rivers-set-take-former-team
http://www.nba.com/celtics/news/sidebar/sullinger-proving-be-steal-nba-draft
https://www.nba.com/celtics/news/sidebar/sullinger-proving-be-steal-nba-draft
https://www.nba.com/celtics/news/sidebar/summer-072718-marcus-smarts-top-5-career-moments
https://www.nba.com/celtics/news/sidebar/summer-081018-2018-19-schedule-breakdown-top-10-games
https://www.nba.com/celtics/news/sidebar/summer-083118-2018-19-roster-breakdown-ball-handlers
https://www.nba.com/celtics/news/sidebar/summer-090618-williams-working-diligently-strengthen-body-and-mind
https://www.nba.com/celtics/news/sidebar/summer-091118-2018-19-roster-breakdown-bigs
https://www.nba.com/celtics/news/sidebar/summer-091318-hayward-reveals-hes-returned-to-5-on-5-play
https://www.nba.com/celtics/news/sideba

In [7]:
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io

from bs4 import BeautifulSoup
domain = "nba.com"
wordsearch = ["celtics","nets","knicks","raptors","76ers"]
index_list = ["2019-04","2019-09","2019-13"]
def search_domain(domain):

    record_list = []
    
    print("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added %d results." % len(records))
            
    
    print("[*] Found a total of %d hits." % len(record_list))
    
    return record_list  

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""
    htmlData = []
    #print(len(data))
    if len(data):
            htmlData = data.decode('UTF-8').strip().split("\r\n\r\n")
            if(len(htmlData)>2):
                response = htmlData[2]
    return response
count=0
weburlTest=[]
mainTopic = "amp"
searchMain = "/"+mainTopic+"/"
record_list = search_domain(domain)
index = open('atlantic_crawlData.txt', 'a+')
for record in record_list:
    for idx in range(0,len(wordsearch)):
            wordsearcht = wordsearch[idx]
            keyword = "/"+wordsearcht+"/"
            if(searchMain.lower() in record['url']):
                if(keyword.lower() in record['url']):
                    print(record['url'])
                    count+=1
                    html_content = download_page(record)
                    soup = BeautifulSoup(html_content)
                    for p in soup.find_all("p", text=True):
                        rete = p.get_text()
                        rete = rete.replace('\n', '')
                        rete = rete.replace('\r\n', '')
                        rete = rete.replace('\\n','')
                        rete = rete.replace('\t','')
                        #print(rete)
                        index.write(rete)
                    index.write("\n")  
print(count)                    

[*] Trying target domain: nba.com
[*] Trying index 2019-04
[*] Added 13445 results.
[*] Trying index 2019-09
[*] Added 14007 results.
[*] Trying index 2019-13
[*] Added 14261 results.
[*] Found a total of 41713 hits.
https://www.nba.com/amp/celtics/video/originals/ainge-trade-deadline-presser-020918
https://www.nba.com/amp/celtics/video/originals/celticsdaily-020918
https://www.nba.com/amp/celtics/video/originals/film-study-baynes-verticality-121917
https://www.nba.com/amp/celtics/video/team/pregame-interview-020818
https://www.nba.com/amp/celtics/video/team/pregame-interview-030718
https://www.nba.com/amp/celtics/video/team/pregame-interview-030818
https://www.nba.com/amp/celtics/video/teams/celtics/2018/02/09/1917205/1518204570875-postgamereport020818atwasver2-1917205
https://www.nba.com/amp/celtics/video/wsc/teams/03d339fe-ca2f-6e0d-0588-11e4b8bea365
https://www.nba.com/amp/celtics/video/wsc/teams/30485d37-1f5a-2f1d-f2c6-1a01ee2a7670
https://www.nba.com/amp/celtics/video/wsc/teams/3

https://www.nba.com/amp/raptors/video/teams/raptors/2019/01/18/2421519/1547786825373-nba-web-190107-cjmiles-sound-v2digital-2421519
https://www.nba.com/amp/raptors/video/teams/raptors/2019/01/18/2421527/1547787100004-nba-web-190107-vanvleet-sounddigital-2421527
https://www.nba.com/amp/raptors/video/teams/raptors/2019/01/20/2424238/1547956900833-nba-web-190119-powell-sounddigital-2424238
https://www.nba.com/amp/raptors/video/teams/raptors/2019/01/20/2424252/1547957197152-nba-web-190119-green-sounddigital-2424252
https://www.nba.com/amp/raptors/video/teams/raptors/2019/01/20/2424258/1547957506559-nba-web-190119-miles-sounddigital-2424258
https://www.nba.com/amp/raptors/video/teams/raptors/2019/01/24/2430011/1548301740886-nba-web-190123-siakampostdigital-2430011
https://www.nba.com/amp/raptors/video/teams/raptors/2019/01/24/2430019/1548301850446-nba-web-190123-greenpostdigital-2430019
https://www.nba.com/amp/raptors/video/teams/raptors/2019/01/24/2430091/1548303297923-nba-web-190123-vanvl

In [8]:
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io

from bs4 import BeautifulSoup
domain = "nba.com"
wordsearch = ["warriors","clippers","lakers","suns","kings"]
index_list = ["2019-04","2019-09","2019-13"]
def search_domain(domain):

    record_list = []
    
    print("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added %d results." % len(records))
            
    
    print("[*] Found a total of %d hits." % len(record_list))
    
    return record_list  

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""
    htmlData = []
    #print(len(data))
    if len(data):
            htmlData = data.decode('UTF-8').strip().split("\r\n\r\n")
            if(len(htmlData)>2):
                response = htmlData[2]
    return response
count=0
weburlTest=[]
mainTopic = "news"
searchMain = "/"+mainTopic+"/"
record_list = search_domain(domain)
index = open('pacific_crawlData.txt', 'w')
for record in record_list:
    for idx in range(0,len(wordsearch)):
            wordsearcht = wordsearch[idx]
            keyword = "/"+wordsearcht+"/"
            if(searchMain.lower() in record['url']):
                if(keyword.lower() in record['url']):
                    print(record['url'])
                    count+=1
                    html_content = download_page(record)
                    soup = BeautifulSoup(html_content)
                    for p in soup.find_all("p", text=True):
                        rete = p.get_text()
                        rete = rete.replace('\n', '')
                        rete = rete.replace('\r\n', '')
                        rete = rete.replace('\\n','')
                        rete = rete.replace('\t','')
                        #print(rete)
                        index.write(rete)
                    index.write("\n")     
print(count)                    

[*] Trying target domain: nba.com
[*] Trying index 2019-04
[*] Added 13445 results.
[*] Trying index 2019-09
[*] Added 14007 results.
[*] Trying index 2019-13
[*] Added 14261 results.
[*] Found a total of 41713 hits.
https://www.nba.com/clippers/news/2018-summer-league-recap
http://www.nba.com/clippers/news/breakingnews091109.html
https://www.nba.com/clippers/news/breakingnews091109.html
https://www.nba.com/clippers/news/breakingnews100204.html
http://www.nba.com/clippers/news/breakingnews_100728.html
https://www.nba.com/clippers/news/breakingnews_100728.html
https://www.nba.com/clippers/news/brian_sieman.html
http://www.nba.com/clippers/news/clippers-trade-evans-nets-120711.html
https://www.nba.com/clippers/news/clippers-trade-evans-nets-120711.html
http://www.nba.com/clippers/news/davis-being-team-fresh-energy
https://www.nba.com/clippers/news/davis-being-team-fresh-energy
http://www.nba.com/clippers/news/dunleavy_030711.html
https://www.nba.com/clippers/news/dunleavy_030711.html
htt

In [9]:
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io

from bs4 import BeautifulSoup
domain = "nba.com"
wordsearch = ["warriors","clippers","lakers","suns","kings"]
index_list = ["2019-04","2019-09","2019-13"]
def search_domain(domain):

    record_list = []
    
    print("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added %d results." % len(records))
            
    
    print("[*] Found a total of %d hits." % len(record_list))
    
    return record_list  

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""
    htmlData = []
    #print(len(data))
    if len(data):
            htmlData = data.decode('UTF-8').strip().split("\r\n\r\n")
            if(len(htmlData)>2):
                response = htmlData[2]
    return response
count=0
weburlTest=[]
mainTopic = "amp"
searchMain = "/"+mainTopic+"/"
record_list = search_domain(domain)
index = open('pacific_crawlData.txt', 'a+')
for record in record_list:
    for idx in range(0,len(wordsearch)):
            wordsearcht = wordsearch[idx]
            keyword = "/"+wordsearcht+"/"
            if(searchMain.lower() in record['url']):
                if(keyword.lower() in record['url']):
                    print(record['url'])
                    count+=1
                    html_content = download_page(record)
                    soup = BeautifulSoup(html_content)
                    for p in soup.find_all("p", text=True):
                        rete = p.get_text()
                        rete = rete.replace('\n', '')
                        rete = rete.replace('\r\n', '')
                        rete = rete.replace('\\n','')
                        rete = rete.replace('\t','')
                        #print(rete)
                        index.write(rete)
                    index.write("\n") 
print(count)

[*] Trying target domain: nba.com
[*] Trying index 2019-04
[*] Added 13445 results.
[*] Trying index 2019-09
[*] Added 14007 results.
[*] Trying index 2019-13
[*] Added 14261 results.
[*] Found a total of 41713 hits.
https://www.nba.com/amp/clippers/clippers-full-court-press-sign
https://www.nba.com/amp/clippers/video/teams/clippers/2018/03/08/1974597/1520539289750-030818-doc-rivers-1974597
https://www.nba.com/amp/clippers/video/teams/clippers/2018/12/12/2375835/1544598806879-181211docpostgame-2375835
https://www.nba.com/amp/clippers/video/teams/clippers/2018/12/12/2376463/1544639906634-los-angeles-clippers-highlights-vs-toronto
https://www.nba.com/amp/clippers/video/teams/clippers/2018/12/13/2377117/1544663996277-version-1episode-1bumblev6-2377117
https://www.nba.com/amp/kings/franchise-record-threes
https://www.nba.com/amp/kings/video/teams/kings/2018/12/13/2377626/1544682598149-bogdanovic-20-points-highlights-vs-min
https://www.nba.com/amp/kings/video/teams/kings/2018/12/13/2377640/

In [10]:
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io

from bs4 import BeautifulSoup
domain = "nba.com"
wordsearch = ["mavericks","rockets","grizzlies","pelicans","spurs"]
index_list = ["2019-04","2019-09","2019-13"]
def search_domain(domain):

    record_list = []
    
    print("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added %d results." % len(records))
            
    
    print("[*] Found a total of %d hits." % len(record_list))
    
    return record_list  

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""
    htmlData = []
    #print(len(data))
    if len(data):
            htmlData = data.decode('UTF-8').strip().split("\r\n\r\n")
            if(len(htmlData)>2):
                response = htmlData[2]
    return response
count=0
weburlTest=[]
mainTopic = "amp"
searchMain = "/"+mainTopic+"/"
record_list = search_domain(domain)
index = open('southwest_crawlData.txt', 'w')
for record in record_list:
    for idx in range(0,len(wordsearch)):
            wordsearcht = wordsearch[idx]
            keyword = "/"+wordsearcht+"/"
            if(searchMain.lower() in record['url']):
                if(keyword.lower() in record['url']):
                    print(record['url'])
                    count+=1
                    html_content = download_page(record)
                    soup = BeautifulSoup(html_content)
                    for p in soup.find_all("p", text=True):
                        rete = p.get_text()
                        rete = rete.replace('\n', '')
                        rete = rete.replace('\r\n', '')
                        rete = rete.replace('\\n','')
                        rete = rete.replace('\t','')
                        #print(rete)
                        index.write(rete)
                    index.write("\n")
print(count)                    

[*] Trying target domain: nba.com
[*] Trying index 2019-04
[*] Added 13445 results.
[*] Trying index 2019-09
[*] Added 14007 results.
[*] Trying index 2019-13
[*] Added 14261 results.
[*] Found a total of 41713 hits.
https://www.nba.com/amp/grizzlies/video/gcm-alexis-all-access-methodist-healthcare-fitness-180201
https://www.nba.com/amp/grizzlies/video/gcm-alexis-jarell-martin-1on1-180209
https://www.nba.com/amp/grizzlies/video/gcm-alexis-jaren-jackson-1on1-181016
https://www.nba.com/amp/grizzlies/video/gcm-alexis-morgan-postup-171222
https://www.nba.com/amp/grizzlies/video/gcm-alexis-morgan-postup-3-180126
https://www.nba.com/amp/grizzlies/video/gcm-chris-vernon-show-180815
https://www.nba.com/amp/grizzlies/video/gcm-chris-vernon-show-evans-180208
https://www.nba.com/amp/grizzlies/video/gcm-chris-vernon-show-on-this-years-nba-draft-pt-1-180209
https://www.nba.com/amp/grizzlies/video/gcm-feat-3v3-corporate-tournament-nike-180227
https://www.nba.com/amp/grizzlies/video/gcm-the-postup-al

https://www.nba.com/amp/pelicans/video/teams/pelicans/2019/01/24/2430125/1548304095112-012319-new-orleans-pelicans-highlights-vs
https://www.nba.com/amp/pelicans/video/teams/pelicans/2019/01/24/2430310/1548317016315-holiday012419-2430310
https://www.nba.com/amp/pelicans/video/teams/pelicans/2019/01/24/2430311/1548317042005-gentry012419-2430311
https://www.nba.com/amp/rockets/video/teams/rockets/2018/02/08/1912783/1518060131520-img3425-1912783
https://www.nba.com/amp/rockets/video/teams/rockets/2018/02/08/1912788/1518060179015-img3427-1912788
https://www.nba.com/amp/rockets/video/teams/rockets/2019/01/17/2419740/1547699030846-coachd-2419740
https://www.nba.com/amp/rockets/video/teams/rockets/2019/01/17/2419744/1547699186122-0bf60751-6512-4d43-b019-bf50a4af57fa-2419744
https://www.nba.com/amp/rockets/video/teams/rockets/2019/01/18/2422188/1547837439060-mda-2422188
https://www.nba.com/amp/rockets/video/teams/rockets/2019/01/18/2422190/1547837610386-untitled-2422190
https://www.nba.com/amp

In [12]:
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io

from bs4 import BeautifulSoup
domain = "nba.com"
search_keywords = ["nuggets","timberwolves","thunder","blazers","jazz"]
index_list = ["2019-04","2019-09","2019-13"]
def search_domain(domain):

    record_list = []
    
    print("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added %d results." % len(records))
            
    
    print("[*] Found a total of %d hits." % len(record_list))
    
    return record_list  

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""
    htmlData = []
    #print(len(data))
    if len(htmlData):
            htmlData = data.decode('UTF-8').strip().split("\r\n\r\n")
            if(len(htmlData)>2):
                response = htmlData[2]
                #print(fdg[0])
                #print(response)
    return response
count=0
weburlTest=[]
category = "news"
keywordtest = "/"+category+"/"
record_list = search_domain(domain)
index = open('northwest_crawlData.txt', 'w')
for record in record_list[:5000]:
    for idx in range(0,len(search_keywords)):
        #if(keywordtest.lower() in record['url']): 
            search_keyword = search_keywords[idx]
            keyword = "/"+search_keyword+"/"
            #print(keyword)
            if(keywordtest.lower() in record['url']):
                    if(keyword.lower() in record['url']):
                        print(record['url'])
                        count+=1
                        html_content = download_page(record)
                        soup = BeautifulSoup(html_content)
                        for p in soup.find_all("p", text=True):
                            rete = p.get_text()
                            rete = rete.replace('\n', '')
                            rete = rete.replace('\r\n', '')
                            rete = rete.replace('\\n','')
                            rete = rete.replace('\t','')
                            #print(rete)
                            index.write(rete)
                    index.write("\n")
print(count)                    

[*] Trying target domain: nba.com
[*] Trying index 2019-04
[*] Added 13445 results.
[*] Trying index 2019-09
[*] Added 14007 results.
[*] Trying index 2019-13
[*] Added 14261 results.
[*] Found a total of 41713 hits.
http://www.nba.com/amp/thunder/news/collison-190112
https://www.nba.com/blazers/news/after-4-1-homestand-blazers-head-denver-face-best-west
https://www.nba.com/blazers/news/blazers-bounce-back-blowout-win-versus-timberwolves
https://www.nba.com/blazers/news/blazers-dominate-hornets-start-finish-extend-win-streak-four
https://www.nba.com/blazers/news/blazers-down-lakers-win-2018-las-vegas-summer-league-tournament-championship
https://www.nba.com/blazers/news/blazers-finish-first-trip-season-tonight-houston
https://www.nba.com/blazers/news/blazers-hold-clippers-third-straight-victory
https://www.nba.com/blazers/news/blazers-host-celtics-last-game-early-season-homestand
https://www.nba.com/blazers/news/blazers-play-host-lakers-saturday-opening-night-rematch
https://www.nba.co

In [13]:
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io

from bs4 import BeautifulSoup
domain = "nba.com"
search_keywords = ["hawks","hornets","heat","magic","wizards"]
index_list = ["2019-04","2019-09","2019-13"]
def search_domain(domain):

    record_list = []
    
    print("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added %d results." % len(records))
            
    
    print("[*] Found a total of %d hits." % len(record_list))
    
    return record_list  

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""
    htmlData = []
    #print(len(data))
    if len(data):
            htmlData = data.decode('UTF-8').strip().split("\r\n\r\n")
            if(len(htmlData)>2):
                response = htmlData[2]
                #print(fdg[0])
                #print(response)
    return response
count=0
weburlTest=[]
category = "amp"
keywordtest = "/"+category+"/"
record_list = search_domain(domain)
index = open('southeast_crawlData.txt', 'w')
for record in record_list:
    for idx in range(0,len(search_keywords)):
        #if(keywordtest.lower() in record['url']): 
            search_keyword = search_keywords[idx]
            keyword = "/"+search_keyword+"/"
            #print(keyword)
            if(keywordtest.lower() in record['url']):
                if(keyword.lower() in record['url']):
                    print(record['url'])
                    count+=1
                    html_content = download_page(record)
                    soup = BeautifulSoup(html_content)
                    for p in soup.find_all("p", text=True):
                        rete = p.get_text()
                        rete = rete.replace('\n', '')
                        rete = rete.replace('\r\n', '')
                        rete = rete.replace('\\n','')
                        rete = rete.replace('\t','')
                        #print(rete)
                        index.write(rete)
                    index.write("\n") 
print(count)                    

[*] Trying target domain: nba.com
[*] Trying index 2019-04
[*] Added 13445 results.
[*] Trying index 2019-09
[*] Added 14007 results.
[*] Trying index 2019-13
[*] Added 14261 results.
[*] Found a total of 41713 hits.
https://www.nba.com/amp/hawks/black-fives-story-atlanta-municipal-auditorium
https://www.nba.com/amp/hawks/collins-with-third-career-high-of-the-season-scores-thirty-against-nuggets
https://www.nba.com/amp/hawks/couple-dating-swipe-right-night-married-arena-video
https://www.nba.com/amp/hawks/deandre-bembry-notches-a-new-career-high-with-eighteen-points
https://www.nba.com/amp/hawks/dennis-schroder-creates-magic-orlando-despite-loss
https://www.nba.com/amp/hawks/hawks-keep-fast-pace-vs-raptors-highlight-video
https://www.nba.com/amp/hawks/hawks-players-make-it-rain-in-philips-arena-cavaliers
https://www.nba.com/amp/hawks/hawks-soar-fashion-thrilling-win-over-grizzlies
https://www.nba.com/amp/hawks/kap-g-episodio-uno-rapper-heritage-college-park
https://www.nba.com/amp/hawk

https://www.nba.com/amp/hawks/trae-young-impresses-notches-double-double
https://www.nba.com/amp/heat/video/dwyane-wade-post-sound-181212
https://www.nba.com/amp/heat/video/erik-spoelstra-post-sound-181212
https://www.nba.com/amp/heat/video/ko-3-181212
https://www.nba.com/amp/hornets/hornets-unveil-statement-uniform-2017-18-season
https://www.nba.com/amp/hornets/video/teams/hornets/2018/03/01/1958875/1519940334223-180301-febdunks-1958875
https://www.nba.com/amp/magic/videos/aaron-gordon/orlando-magic-highlights-clutch-shot-los-angeles-lakers-20180307
93


In [14]:
import requests
import argparse
import time
import json
import gzip
import csv
import codecs
import io

from bs4 import BeautifulSoup
domain = "nba.com"
search_keywords = ["bulls","cavaliers","pistons","pacers","bucks"]
index_list = ["2019-04","2019-09","2019-13"]
def search_domain(domain):

    record_list = []
    
    print("[*] Trying target domain: %s" % domain)
    
    for index in index_list:
        
        print("[*] Trying index %s" % index)
        
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&matchType=domain&output=json" % domain
        
        response = requests.get(cc_url)
        
        if response.status_code == 200:
            
            records = response.content.splitlines()
            
            for record in records:
                record_list.append(json.loads(record))
            
            print("[*] Added %d results." % len(records))
            
    
    print("[*] Found a total of %d hits." % len(record_list))
    
    return record_list  

def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'
    
    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})
    
    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = io.BytesIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)
    
    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""
    fdg = []
    #print(len(data))
    if len(data):
            htmlData = data.decode('UTF-8').strip().split("\r\n\r\n")
            if(len(htmlData)>2):
                response = htmlData[2]
                #print(fdg[0])
                #print(response)
    return response
count=0
weburlTest=[]
category = "news"
keywordtest = "/"+category+"/"
record_list = search_domain(domain)
index = open('central_crawlData.txt', 'w')
for record in record_list[5000:7100]:
    for idx in range(0,len(search_keywords)):
        #if(keywordtest.lower() in record['url']): 
            search_keyword = search_keywords[idx]
            keyword = "/"+search_keyword+"/"
            #print(keyword)
            if(keywordtest.lower() in record['url']):
                if(keyword.lower() in record['url']):
                    print(record['url'])
                    count+=1
                    html_content = download_page(record)
                    soup = BeautifulSoup(html_content)
                    for p in soup.find_all("p", text=True):
                        rete = p.get_text()
                        #print(rete)
                        index.write(rete)
                    index.write("\n") 
print(count)                    

[*] Trying target domain: nba.com
[*] Trying index 2019-04
[*] Added 13445 results.
[*] Trying index 2019-09
[*] Added 14007 results.
[*] Trying index 2019-13
[*] Added 14261 results.
[*] Found a total of 41713 hits.
https://www.nba.com/bucks/news/bucks-acquire-future-2nd-round-draft-pick-and-cash-washington
http://www.nba.com/bucks/news/bucks-assign-sterling-brown-wisconsin-herd
https://www.nba.com/bucks/news/bucks-assign-sterling-brown-wisconsin-herd
https://www.nba.com/bucks/news/bucks-exercise-rookie-scale-options-thon-maker-and-dj-wilson
https://www.nba.com/bucks/news/bucks-froedtert-mcw-health-network-welcome-newborns-2019-keepsake-blanket
https://www.nba.com/bucks/news/bucks-organization-statement-sterling-brown
https://www.nba.com/bucks/news/bucks-recall-dj-wilson-wisconsin-herd-1
https://www.nba.com/bucks/news/bucks-request-waivers-mirza-teletovic
https://www.nba.com/bucks/news/bucks-sign-brandon-jennings-second-10-day-contract
https://www.nba.com/bucks/news/fiserv-forum-opens

https://www.nba.com/bulls/news/bulls-look-refocus-rebound-ahead-west-coast-road-trip
http://www.nba.com/bulls/news/bulls-lose-grizzlies-98-91
https://www.nba.com/bulls/news/bulls-lose-grizzlies-98-91
https://www.nba.com/bulls/news/bulls-lose-pelicans-107-98-despite-strong-rally-fourth
http://www.nba.com/bulls/news/bulls-lost-might-have-found-more-talent
https://www.nba.com/bulls/news/bulls-lost-might-have-found-more-talent
http://www.nba.com/bulls/news/bulls-orlando-hoping-derrick-roses-return
https://www.nba.com/bulls/news/bulls-orlando-hoping-derrick-roses-return
https://www.nba.com/bulls/news/bulls-practice-incident
http://www.nba.com/bulls/news/bulls-re-sign-kirk-hinrich/
https://www.nba.com/bulls/news/bulls-re-sign-kirk-hinrich/
https://www.nba.com/bulls/news/bulls-retain-lavine
https://www.nba.com/bulls/news/bulls-sign-antonio-blakeney
https://www.nba.com/bulls/news/bulls-sign-carter-jr-and-hutchison
https://www.nba.com/bulls/news/bulls-sign-derrick-walton-jr
http://www.nba.com/b